라이브러리들

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage,HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler



자주 쓰는 변수들

In [ ]:
CHAT = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#3.0

In [ ]:
a = CHAT.predict("좀 있다가 에슐리 퀸즈를 가는데, 먹을거 추천 좀 해줘봐")
# 예제에서 특징은 chat이라는 변수에 어떠한 모델을 사용하든, predict라는 메소드 사용 가능 / 답변은 매번 바뀜

a

#3.1

In [ ]:
message = [
    SystemMessage(
        content="넌 이제부터 매우 awsome하고 coooool한 음식의 specialist야. 그 어떠한 음식에 대해 물어보면 재료와 레시피에 대해 말할 수 있어."),
    AIMessage(
        content="안녕하세요. 저는 맛있는 녀석입니다."),
    HumanMessage(
        content="일식 덮밥집에서 파는 부타동에 대해 설명해줘.")
]

CHAT.predict_messages(message)

#3.2

In [ ]:
prom_template = PromptTemplate.from_template("{place}을(를) 가는데, 추천 좀 해줘봐")
#PromptTemplate는 단순 메세지를 프롬프트로 사용 할 때 유용함

prom_template = prom_template.format(place = "맘스터치")

CHAT.predict(prom_template)

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "넌 이제부터 매우 awsome하고 coooool한 {job}의 specialist야. 그 어떠한 질문이든 대답 할 수 있어."),
    ("ai", "안녕하세요. 저는 {name}입니다."),
    ("human", "{place}을(를) 가는데, 추천 좀 해줘."),
])
#ChatPromptTemplate는 조금 더 정교하게 설정된 프롬프트를 탬플릿을 통해 사용 할 때 유용함


chat_template = chat_template.format(job="음식",name="맛있는 녀석",place="맘스터치")

CHAT.predict(chat_template)

#3.3

In [ ]:
#BaseOutputParser를 사용하여 결과 출력을 parse하기

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))
    
parser = CommaOutParser()
parser.parse("  안녕하세요     .   유령개입니다 .    잘부탁드리겠습니다.")

#3분 24초

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])

chat_template = chat_template.format(
    n = 10,
    question = "아시아에 위치한 나라를 알려주세요.  "
)

rs = CHAT.predict(chat_template)
parser.parse(rs)

#-------------------------

In [ ]:
# 위의 두개와 달리, 더 간단하게 할 수 있는 버전이 있다. 여기선 오직 CHAT이라는 chatmodel 변수와, template. parser 세 개만이 존재한다.

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))

chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])


chain = chat_template | CHAT | CommaOutParser()
chain.invoke({
    "n":3,
    "question":"유럽의 나라"
})

#3.4

In [ ]:
chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 슈퍼 클래스의 요리사입니다. 어떠한 요리를 요구하든 그 요리의 재료와 레시피에 대해 알려줄 수 있습니다."),
    ("human", "저는 {food} 요리를 만들고 싶습니다. 들어가는 재료와 레시피에 대해 알려주실 수 있을까요?"),
])
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 어떠한 요리의 레시피가 들어와도 채식으로 변형시킬 수 있는 세계 대표의 채식 요리사입니다. 레시피가 들어오면, 해당 메뉴의 재료를 채식에 맞게 변형시켜야합니다."),
    ("human", "{recipe}"),
])

chef_chain = chef_template | CHAT
veg_chef_chain = veg_chef_template | CHAT
fianal_chain = {"recipe":chef_chain} | veg_chef_chain # chef_template | CHAT | veg_chef_template | CHAT << 이것과 같은 형태임.
# 여러개의 체인을 위와 같은 방식으로 엮을 수 있음. 이건 머신러닝과 엇비슷한 형태라고 볼 수 있음

fianal_chain.invoke({
    "food": "프랑스"
})


#4.0